# Traiter la donnée -- Dataframes


Pour le réaliser sans installation, depuis un navigateur :
<a href="https://colab.research.google.com/github/eddes/INSA/blob/main/python/tuto_dataframe.ipynb"> ça se passe ici<a>


Dans de nombreux des domaines, la donnée disponible est fournie dans des fichiers texte, où les valeurs des différentes grandeurs sont séparées par des virgules (vos tableurs favoris exporte parfois en `.csv` - acronyme de _comma separated values_.
    
Une librairie particulièrement bien adaptée au traitement de tels fichiers est `pandas`. Dans ce qui suit, nous allons apprendre les bases 
    
## Ouverture et affichage de base

Ouvrons un des fichiers de température de la ville de Strasbourg.

In [1]:
import pandas as pd
chemin='./src/horaire_Strasbourg_2016.csv'
donnee = pd.read_csv(chemin)

Pour afficher quelques stats de base :

In [3]:
donnee.describe()

,temp
count,8643.000000
mean,11.077982
std,7.971636
min,-6.700000
25%,5.000000
50%,10.300000
75%,16.900000
max,35.900000


On peut donc lire les min/max ainsi que les percentiles de la série.

Ensuite, il est pratique de connaître le nom des données stockées :

In [4]:
donnee.keys()

Index(['time', 'temp'], dtype='object')

On constate que le temps et la température sont stockées. On va pouvoir y faire appel simplement par leur nom avec une commande du style :

In [5]:
donnee['temp']

0       4.7
1       4.4
2       4.8
3       4.5
4       3.8
       ... 
8641   -2.0
8642   -1.8
8643   -1.9
8644   -2.0
8645   -2.0
Name: temp, Length: 8646, dtype: float64

On notera la présence d'un index (colonne de gauche qui est affichée). Cet index est commun à toutes les données stockées, lorsqu'il y en a plusieurs. Il peut aussi être un temps (heure/min/sec j/m/a) ce qui se révèlera pratique par la suite.

Si on souhaite uniquement prendre les valeurs :

In [8]:
T = donnee['temp'].values
print(T)

[ 4.7  4.4  4.8 ... -1.9 -2.  -2. ]
